Notebook showing how to change power settings while writing miner data to SQLite db

In [7]:
import asyncio
from pyasic import get_miner
from pyasic_tools.db import (
    load_db,
    write_data,
)

In [8]:

async def change_power(miner, sleep_mins: int) -> None:
    await miner.set_power_limit(300)
    await asyncio.sleep(sleep_mins * 60)
    await miner.set_power_limit(600)
    await asyncio.sleep(sleep_mins * 60)
    await miner.set_power_limit(900)
    await asyncio.sleep(sleep_mins * 60)

async def run_tasks(miner, db_file):
    write_task = asyncio.create_task(write_data(miner=miner, data_file=db_file, sleep_mins=1))
    power_task = asyncio.create_task(change_power(miner=miner, sleep_mins=30))
    
    done, pending = await asyncio.wait({write_task, power_task}, return_when=asyncio.FIRST_COMPLETED)
    
    if power_task in done:
        for task in pending:
            task.cancel()
            try:
                await task
            except asyncio.CancelledError:
                pass



In [ ]:

# change power every 30 mins while writing data to SQLite db
ip = "192.168.0.164"
db_file = "change_power.db"
miner = await get_miner(ip)
await run_tasks(miner, db_file)

In [9]:
# load SQLite db into pandas dataframe
df_db = load_db(db_file="change_power.db", table_name="data")
df_db.head()

,ip,datetime,uptime,model,make,api_ver,fw_ver,hostname,hashrate,expected_hashrate,...,hashboard_2_slot,hashboard_2_hashrate,hashboard_2_temp,hashboard_2_chip_temp,hashboard_2_chips,hashboard_2_expected_chips,hashboard_2_serial_number,hashboard_2_missing,fan_0_speed,fan_1_speed
0,192.168.0.164,2024-03-01 06:47:22.897163-05:00,80988,S9 (BOS+),AntMiner,3.7,22.08.1,miner-739016,3.34,3.33,...,2,1.12,38.0,53.0,63.0,63,None,0,1440,900
1,192.168.0.164,2024-03-01 06:48:25.248565-05:00,60,S9 (BOS+),AntMiner,3.7,22.08.1,miner-739016,2.55,3.33,...,2,0.84,32.0,47.0,63.0,63,None,0,1440,900
2,192.168.0.164,2024-03-01 06:49:27.426364-05:00,122,S9 (BOS+),AntMiner,3.7,22.08.1,miner-739016,3.35,3.33,...,2,1.11,36.0,51.0,63.0,63,None,0,1440,900
3,192.168.0.164,2024-03-01 06:50:29.836579-05:00,185,S9 (BOS+),AntMiner,3.7,22.08.1,miner-739016,3.34,3.33,...,2,1.12,41.0,56.0,63.0,63,None,0,1440,900
4,192.168.0.164,2024-03-01 06:51:33.385691-05:00,248,S9 (BOS+),AntMiner,3.7,22.08.1,miner-739016,3.18,3.33,...,2,1.07,34.0,49.0,63.0,63,None,0,1440,900
